In [1]:
%pip install transformers
%pip install datasets
%pip install torch 
%pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric
import torch
import numpy as np

In [3]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher", "/data") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Using custom data configuration bypublisher-data_dir=%2Fdata
Reusing dataset hyperpartisan_news_detection (/root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)


  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['train', 'validation'])

In [4]:
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = 32)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-a210232c66df418e.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-83a0000c6d2b7eb5.arrow


In [7]:
int_labels_train = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]
int_labels_val = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]

In [6]:
xl_net_model = XLNetForSequenceClassification.from_pretrained("xlnet-large-cased", labels = 2)

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [5]:
from transformers import XLNetTokenizer

xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")

In [8]:
f1_score = load_metric("f1")

def xl_tokenize(batch):
    return xlnet_tokenizer(batch['text'])

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return f1_score.compute(predictions = predictions, references = label)

def hp_space(trial):

  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.0001,0.0005, 0.0001)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)
  }

def xlnet_init():
  

In [ ]:
hyperpartisan_train = xlnet_tokenizer(hyperpartisan_train['text'])
hyperpartisan_valid = xlnet_tokenizer(hyperpartisan_valid['text'])

WITH FINETUNING

USING kfold FOR Cross validation

In [9]:
from sklearn.model_selection import KFold

fold = KFold(n_splits = 5)
this_set = fold.split(hyperpartisan_train)

In [12]:
for i,j in this_set:
  training_set = hyperpartisan_train.select(range(i))
  eval_set = hyperpartisan_train.select(range(j))

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=xlnet_tokenizer)

#default training arguments 
training_args = TrainingArguments(output_dir="./results", learning_rate=2e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8, num_train_epochs=5,weight_decay=0.01)

#fine-tune
trainer = Trainer(model = xl_net_model, args = training_args,
            train_dataset= training_set,
            eval_dataset= eval_set,
            tokenizer= xlnet_tokenizer,
            data_collator=data_collator,
            compute_metrics= compute_metrics)

trainer.train()

In [ ]:
#Obtaining best hyperparameter setting, then finetuning based on the number of files.